# 55 Public, Cleveland Simulations
8760 rows per simulation<br>
Floors for residential and commercial

## Preprocess

1. Generate zone with facade glazing systems in COMFEN.

2. Retrieve IDFs for each scenario from the COMFEN5 Results folder. Have to do each of the 3 sections of the building as its own sqlite file and each project by orientation separately. Then IDF files are copied after each simulation. These files will be used in the Jupyter Notebook.

3. Convert from E+ 8.1 to 9.5 using the .lst format.
4. Change the reporting in the IDF files to "All" to get the HTML summary table and the inch-pound units for verification.
      
    * OutputControl:Table:Style,
    * All,                     !- Column Separator
    * InchPound;               !- Unit Conversion
5. Rerun batch sims in EnergyPlus v9.5.
6. Add "Scenario", "Elevation, and "GlazingSystem" to the results CSV files.

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

## Energy Costs and Emission Factors

http://seeatcalc.gastechnology.org/ResidentialBuildings.aspx

In [2]:
# Costs (elec $/kWh, gas $/therm), change values per city
resgas_cost = 1.16
reselec_cost = 0.1238

comgas_cost = 0.56
comelec_cost = 0.0972

In [3]:
# Source Energy Conversion Factors (same for res or comm), change values per city
gas_source = 1.09
elec_source = 2.93

In [4]:
# Carbon equivalent factors (same for res or comm), change values per city

gas_co2 = 0.14566
elec_co2 = 0.42103

In [5]:
# Zone size for EUI

sf = 329.55

In [6]:
# Lighting energy reduction, assumption that LEDs are being used. 
# Use this multiplier if reductions are not in the COMFEN model

lighting_redux = 1

In [7]:
# # For peak demand calculations (.0002931 kW = 1 Btu).

# kw = 0.0002931

---

## Combine CSV Files in Directory Residential Electric Heating

In [8]:
# Create variable for files in directory
files = [f for f in os.listdir("data/cle/ResElecElec/") if f.endswith(".csv")]

# files

In [9]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_ResElec = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/cle/ResElecElec/" + file)
    all_ResElec = pd.concat([all_ResElec, df])
    
all_ResElec

# 105120  rows, 108 columns

,Scenario,Elevation,GlazingSystem,Date/Time,Environment:Site Outdoor Air Drybulb Temperature [C](Hourly),COMFENZONE PEOPLE 1:People Occupant Count [](Hourly),COMFENZONE:Zone Lights Electricity Energy [J](Hourly),COMFENZONE:Zone Windows Total Heat Gain Rate [W](Hourly),COMFENZONE:Zone Windows Total Heat Gain Energy [J](Hourly),EXTWALL:Surface Outside Face Sunlit Fraction [](Hourly),...,COMFENZONE PEOPLE 1:Zone Thermal Comfort Fanger Model PPD [%](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model 90% Acceptability Status [](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model 80% Acceptability Status [](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model Running Average Outdoor Air Temperature [C](Hourly),COMFENZONE PEOPLE 1:Zone Thermal Comfort ASHRAE 55 Adaptive Model Temperature [C](Hourly),COMFENZONE:Zone Cooling Setpoint Not Met Time [hr](Hourly),SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly),SYS-1:Air System NaturalGas Energy [J](Hourly),SYS-1:Air System Fan Electricity Energy [J](Hourly),SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly)
0,cle_res_elec_10,E,Exist_LowE,01/01 01:00:00,-1.2000,1.500,15904.72675,0.0,0.0,0.0,...,16.045265,-1.0,-1.0,-0.886806,-1.0,0.0,3565848.557,0,7.151005e+05,0.0
1,cle_res_elec_10,E,Exist_LowE,01/01 02:00:00,-0.6000,1.500,15904.72675,0.0,0.0,0.0,...,15.885338,-1.0,-1.0,-0.886806,-1.0,0.0,3506993.759,0,7.151005e+05,0.0
2,cle_res_elec_10,E,Exist_LowE,01/01 03:00:00,-0.6000,1.500,15904.72675,0.0,0.0,0.0,...,15.917425,-1.0,-1.0,-0.886806,-1.0,0.0,3588966.686,0,7.151005e+05,0.0
3,cle_res_elec_10,E,Exist_LowE,01/01 04:00:00,-0.6000,1.500,15904.72675,0.0,0.0,0.0,...,15.916149,-1.0,-1.0,-0.886806,-1.0,0.0,3633011.978,0,7.151005e+05,0.0
4,cle_res_elec_10,E,Exist_LowE,01/01 05:00:00,-0.6000,1.500,44390.80451,0.0,0.0,0.0,...,15.914225,-1.0,-1.0,-0.886806,-1.0,0.0,3653283.528,0,7.151005e+05,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,cle_res_elec_09,N,Exist_LowE,12/31 20:00:00,3.4875,1.305,234060.60560,0.0,0.0,0.0,...,15.034123,-1.0,-1.0,-0.714479,-1.0,0.0,1292215.296,0,1.340602e+06,0.0
8756,cle_res_elec_09,N,Exist_LowE,12/31 21:00:00,3.5250,1.305,237383.98130,0.0,0.0,0.0,...,14.801551,-1.0,-1.0,-0.714479,-1.0,0.0,1336613.604,0,1.340602e+06,0.0
8757,cle_res_elec_09,N,Exist_LowE,12/31 22:00:00,3.6750,1.500,164269.71510,0.0,0.0,0.0,...,14.603887,-1.0,-1.0,-0.714479,-1.0,0.0,1348046.103,0,1.340602e+06,0.0
8758,cle_res_elec_09,N,Exist_LowE,12/31 23:00:00,4.2125,1.500,91155.44883,0.0,0.0,0.0,...,14.413026,-1.0,-1.0,-0.714479,-1.0,0.0,1412030.685,0,1.340602e+06,0.0


In [10]:
# Get wanted columns
all_ResElec1 = all_ResElec[["Scenario",
               "Date/Time",
                "Elevation",
               "GlazingSystem",
              "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)",
              "SYS-1:Air System NaturalGas Energy [J](Hourly)",
              "SYS-1:Air System Fan Electricity Energy [J](Hourly)",
              "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ",
             "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)"]]

#all_ComElec1

In [11]:
all_ResElec1 = all_ResElec1.dropna().reset_index(drop=True)

#all_ResElec1

In [12]:
# # Export grouped file to csv
# all_ResElec1.to_csv("Scraper_Output/cle_res_all.csv", header=True, index=True)

-----

## Get the Wanted Data (columns)

In [13]:
# Rename Columns
all_ResElec2 = all_ResElec1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "SYS-1:Air System Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "SYS-1:Air System NaturalGas Energy [J](Hourly)":"HeatingGasEnergy[J](Hourly)",
                            "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)":"HeatingElecEnergy[J](Hourly)",
                            "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly)",
                            "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)":"LightingEnergy[J](Hourly)"
                             })

#all_ResElec2

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>

In [14]:
# Copy dataframe to modify and leave original df intact
get_annual_ResElec = all_ResElec2.copy(deep=True)

#get_annual_ResElec

In [15]:
# Convert Joules to KBtu in dataframe
get_annual_ResElec["FanEnergy(kBtu)"] = get_annual_ResElec["FanEnergy[J](Hourly)"] * .00000094781712
get_annual_ResElec["HeatingElecEnergy(kBtu)"] = get_annual_ResElec["HeatingElecEnergy[J](Hourly)"] * .00000094781712
get_annual_ResElec["HeatingGasEnergy(kBtu)"] = get_annual_ResElec["HeatingGasEnergy[J](Hourly)"] * .00000094781712
get_annual_ResElec["CoolingEnergy(kBtu)"] = get_annual_ResElec["CoolingEnergy[J](Hourly)"] * .00000094781712

# Reduce lighting energy, assuming LEDs are being used
get_annual_ResElec["LightingEnergy(kBtu)"] = (get_annual_ResElec["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux


#get_annual_ComElec

In [16]:
# Drop columns
get_annual_ResElec.drop(columns = ["FanEnergy[J](Hourly)", "HeatingElecEnergy[J](Hourly)", "HeatingGasEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

#get_annual_ResElec

In [17]:
# Add energy totals
totalResEnergy = get_annual_ResElec["FanEnergy(kBtu)"] + get_annual_ResElec["HeatingElecEnergy(kBtu)"] + get_annual_ResElec["LightingEnergy(kBtu)"] + get_annual_ResElec["CoolingEnergy(kBtu)"]
        
get_annual_ResElec["AnnualEnergy(kBtu)"] = totalResEnergy

# get_annual_ResElec

-----

## Get Energy per sf 
Add EUI (site) column<br>

In [18]:
# Get data per sf (energy/sf)
get_annual_ResElec["FanEnergy(kBtu/sf)"] = (get_annual_ResElec["FanEnergy(kBtu)"]) / sf
get_annual_ResElec["HeatingElecEnergy(kBtu/sf)"] = (get_annual_ResElec["HeatingElecEnergy(kBtu)"]) / sf
get_annual_ResElec["HeatingGasEnergy(kBtu/sf)"] = (get_annual_ResElec["HeatingGasEnergy(kBtu)"]) / sf
get_annual_ResElec["CoolingEnergy(kBtu/sf)"] = (get_annual_ResElec["CoolingEnergy(kBtu)"]) / sf
get_annual_ResElec["LightingEnergy(kBtu/sf)"] = (get_annual_ResElec["LightingEnergy(kBtu)"]) / sf

In [19]:
# Establish EUI (energy/sf)
get_annual_ResElec["EUI(kBtu/sf)"] = (get_annual_ResElec["FanEnergy(kBtu)"] +
                                    get_annual_ResElec["HeatingElecEnergy(kBtu)"] +
                                    get_annual_ResElec["LightingEnergy(kBtu)"] +
                                    get_annual_ResElec["CoolingEnergy(kBtu)"]) / sf
                                                       
#get_annual_ResElec

-----

## Get Carbon per sf 
Add carbon/sf column

In [20]:
# Do carbon calculation using variable (lb/sf)
get_annual_ResElec["CO2_Electric(lb/sf)"] = ((get_annual_ResElec["FanEnergy(kBtu)"] +
                                    get_annual_ResElec["HeatingElecEnergy(kBtu)"] +
                                    get_annual_ResElec["LightingEnergy(kBtu)"] +
                                    get_annual_ResElec["CoolingEnergy(kBtu)"]) * elec_co2) / sf
                                                       
#get_annual_ResElec

In [21]:
get_annual_ResElec["CO2_Gas(lb/sf)"] = ((get_annual_ResElec["HeatingGasEnergy(kBtu)"]) * gas_co2) / sf
                                                       
# get_annual_ComElec

---

## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier<br>
Add cost multiplier, variables defined above

In [22]:
# Get annual costs
get_annual_costs_ResElec = all_ResElec2.copy(deep=True)

In [23]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs_ResElec["FanCost($)"] = (get_annual_costs_ResElec["FanEnergy[J](Hourly)"] * .000000277777778) * reselec_cost
get_annual_costs_ResElec["HeatingElecCost($)"] = ((get_annual_costs_ResElec["HeatingElecEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * resgas_cost
get_annual_costs_ResElec["HeatingGasCost($)"] = ((get_annual_costs_ResElec["HeatingGasEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * resgas_cost
get_annual_costs_ResElec["CoolingCost($)"] = (get_annual_costs_ResElec["CoolingEnergy[J](Hourly)"] * .000000277777778) * reselec_cost

# Reduce lighting energy, assuming LEDs are being used
get_annual_costs_ResElec["LightingCost($)"] = (get_annual_costs_ResElec["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux

#get_annual_costs

In [24]:
# Drop columns
get_annual_costs_ResElec.drop(columns = ["FanEnergy[J](Hourly)", "HeatingElecEnergy[J](Hourly)", "HeatingGasEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

# get_annual_costs_ResElec

In [25]:
# Add cost totals
totalResCost = get_annual_costs_ResElec["FanCost($)"] + get_annual_costs_ResElec["HeatingElecCost($)"] + get_annual_costs_ResElec["HeatingGasCost($)"]  + get_annual_costs_ResElec["CoolingCost($)"] + get_annual_costs_ResElec["LightingCost($)"]
        
get_annual_costs_ResElec["AnnualCost($)"] = totalResCost

# get_annual_costs_ResElec

-----

## Get Annual Cost per sf Data
Add costs/sf columns<br>

In [26]:
# Get data per sf ($/sf)
get_annual_costs_ResElec["FanCost($/sf)"] = (get_annual_costs_ResElec["FanCost($)"]) / sf
get_annual_costs_ResElec["HeatingElecCost($/sf)"] = (get_annual_costs_ResElec["HeatingElecCost($)"]) / sf
get_annual_costs_ResElec["HeatingGasCost($/sf)"] = (get_annual_costs_ResElec["HeatingGasCost($)"]) / sf
get_annual_costs_ResElec["CoolingCost($/sf)"] = (get_annual_costs_ResElec["CoolingCost($)"]) / sf
get_annual_costs_ResElec["LightingCost($/sf)"] = (get_annual_costs_ResElec["LightingCost($)"]) / sf

# get_annual_costs_ResElec

In [27]:
# Establish costs ($/sf)
get_annual_costs_ResElec["AnnualCost($/sf)"] = (get_annual_costs_ResElec["FanCost($)"] +
                                    get_annual_costs_ResElec["HeatingElecCost($)"] +
                                    get_annual_costs_ResElec["HeatingGasCost($)"] +  
                                    get_annual_costs_ResElec["CoolingCost($)"] +  
                                    get_annual_costs_ResElec["LightingCost($)"]) / sf
                                                       
# get_annual_costs_ResElec

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [28]:
# Combine data for energy and cost
allAnnualDataResElec = pd.merge(get_annual_ResElec, get_annual_costs_ResElec)

#allAnnualDataResElec

In [29]:
# list(allAnnualDataResElec.columns.values)

---
## Group and Export Annual Data
<!-- Group dataframe by scenario<br> -->
Export to file

In [30]:
# Group by scenario
annualDataRes = allAnnualDataResElec.groupby(["Scenario", "Elevation", "GlazingSystem"], as_index=True)

# annualDataRes.sum().round(2)

In [31]:
annualDataResFinal = annualDataRes.sum().round(2)

# annualDataResFinal

In [32]:
# Export grouped file to csv
annualDataResFinal.to_csv("Scraper_Output/cle_annual_res.csv", header=True, index=True)

---

## Establish Annual Baseline at Existing

Create dataset with just baselines.<br> 
Keep the units in kBtus to be able to eventually combine heating and cooling <br>
Merge the baseline dataset with the full dataset.<br>
Determine the delta between scenario and base.

In [33]:
# Define the scenario for the baselines.
# % improvement from Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees)
baselineAnnualDataResElec = allAnnualDataResElec[allAnnualDataResElec["GlazingSystem"] == "Exist"]

# baselineAnnualDataResElec

In [34]:
# Rename columns to indicate baseline to prepare for merge"

allBaselineAnnualDataResElec = baselineAnnualDataResElec.rename(columns={"Scenario":"B_Scenario",
 'Date_Time':'B_Date_Time',
 'Elevation':'B_Elevation',
 'GlazingSystem':'B_GlazingSystem',
 'FanEnergy(kBtu)':'B_FanEnergy(kBtu)',
 'HeatingElecEnergy(kBtu)':'B_HeatingElecEnergy(kBtu)',
 'HeatingGasEnergy(kBtu)':'B_HeatingGasEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'B_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'B_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'B_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'B_FanEnergy(kBtu/sf)',
 'HeatingElecEnergy(kBtu/sf)':'B_HeatingElecEnergy(kBtu/sf)',
 'HeatingGasEnergy(kBtu/sf)':'B_HeatingGasEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'B_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'B_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'B_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'B_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'B_CO2_Gas(lb/sf)',
 'FanCost($)':'B_FanCost($)',
 'HeatingElecCost($)':'B_HeatingElecCost($)',
 'HeatingGasCost($)':'B_HeatingGasCost($)',
 'CoolingCost($)':'B_CoolingCost($)',
 'LightingCost($)':'B_LightingCost($)',
 'AnnualCost($)':'B_AnnualCost($)',
 'FanCost($/sf)':'B_FanCost($/sf)',
 'HeatingElecCost($/sf)':'B_HeatingElecCost($/sf)',
 'HeatingGasCost($/sf)':'B_HeatingGasCost($/sf)',
 'CoolingCost($/sf)':'B_CoolingCost($/sf)',
 'LightingCost($/sf)':'B_LightingCost($/sf)',
 'AnnualCost($/sf)':'B_AnnualCost($/sf)'
                            })
# allBaselineAnnualDataResElec

---
## Group and Export Baseline Annual Data
Group dataframe by scenario<br>
Export to file

In [35]:
# Group by scenario
annualBaselineDataRes = allBaselineAnnualDataResElec.groupby(["B_Scenario", "B_Elevation", "B_GlazingSystem"], as_index=True)

# annualDataRes.sum().round(2)

In [36]:
annualBaselineDataRes = annualBaselineDataRes.sum().round(2)

# annualDataResFinal

In [37]:
# list(AnnualWithBaselineRes.columns.values)

In [38]:
# Export grouped file to csv
annualBaselineDataRes.to_csv("Scraper_Output/cle_baseline_annual_res.csv", header=True, index=True)

## Find Peak Demand
Slice data to find peak demand for each scenario<br>
Export to csv

In [39]:
# Copy dataframe to modify and leave original df intact
get_peak_ResElec = all_ResElec2.copy(deep=True)

get_peak_ResElec

,Scenario,Date_Time,Elevation,GlazingSystem,HeatingElecEnergy[J](Hourly),HeatingGasEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),LightingEnergy[J](Hourly)
0,cle_res_elec_10,01/01 01:00:00,E,Exist_LowE,3565848.557,0,7.151005e+05,0.0,15904.72675
1,cle_res_elec_10,01/01 02:00:00,E,Exist_LowE,3506993.759,0,7.151005e+05,0.0,15904.72675
2,cle_res_elec_10,01/01 03:00:00,E,Exist_LowE,3588966.686,0,7.151005e+05,0.0,15904.72675
3,cle_res_elec_10,01/01 04:00:00,E,Exist_LowE,3633011.978,0,7.151005e+05,0.0,15904.72675
4,cle_res_elec_10,01/01 05:00:00,E,Exist_LowE,3653283.528,0,7.151005e+05,0.0,44390.80451
...,...,...,...,...,...,...,...,...,...
105115,cle_res_elec_09,12/31 20:00:00,N,Exist_LowE,1292215.296,0,1.340602e+06,0.0,234060.60560
105116,cle_res_elec_09,12/31 21:00:00,N,Exist_LowE,1336613.604,0,1.340602e+06,0.0,237383.98130
105117,cle_res_elec_09,12/31 22:00:00,N,Exist_LowE,1348046.103,0,1.340602e+06,0.0,164269.71510
105118,cle_res_elec_09,12/31 23:00:00,N,Exist_LowE,1412030.685,0,1.340602e+06,0.0,91155.44883


In [40]:
# Add energy totals
# divide by the number of seconds in an hour (3600)
totalResPeak = (get_peak_ResElec["FanEnergy[J](Hourly)"] + get_peak_ResElec["HeatingElecEnergy[J](Hourly)"] + get_peak_ResElec["HeatingGasEnergy[J](Hourly)"] + get_peak_ResElec["CoolingEnergy[J](Hourly)"] + get_peak_ResElec["LightingEnergy[J](Hourly)"])
        
get_peak_ResElec["PeakEnergy[J](Hourly)"] = totalResPeak

get_peak_ResElec["PeakEnergy(w)"] = (get_peak_ResElec["PeakEnergy[J](Hourly)"] / 3600)

get_peak_ResElec["PeakEnergy(w/sf)"] = (get_peak_ResElec["PeakEnergy(w)"] / sf)

get_peak_ResElec

,Scenario,Date_Time,Elevation,GlazingSystem,HeatingElecEnergy[J](Hourly),HeatingGasEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),LightingEnergy[J](Hourly),PeakEnergy[J](Hourly),PeakEnergy(w),PeakEnergy(w/sf)
0,cle_res_elec_10,01/01 01:00:00,E,Exist_LowE,3565848.557,0,7.151005e+05,0.0,15904.72675,4.296854e+06,1193.570503,3.621819
1,cle_res_elec_10,01/01 02:00:00,E,Exist_LowE,3506993.759,0,7.151005e+05,0.0,15904.72675,4.237999e+06,1177.221948,3.572210
2,cle_res_elec_10,01/01 03:00:00,E,Exist_LowE,3588966.686,0,7.151005e+05,0.0,15904.72675,4.319972e+06,1199.992206,3.641305
3,cle_res_elec_10,01/01 04:00:00,E,Exist_LowE,3633011.978,0,7.151005e+05,0.0,15904.72675,4.364017e+06,1212.227009,3.678431
4,cle_res_elec_10,01/01 05:00:00,E,Exist_LowE,3653283.528,0,7.151005e+05,0.0,44390.80451,4.412775e+06,1225.770795,3.719529
...,...,...,...,...,...,...,...,...,...,...,...,...
105115,cle_res_elec_09,12/31 20:00:00,N,Exist_LowE,1292215.296,0,1.340602e+06,0.0,234060.60560,2.866878e+06,796.354900,2.416492
105116,cle_res_elec_09,12/31 21:00:00,N,Exist_LowE,1336613.604,0,1.340602e+06,0.0,237383.98130,2.914599e+06,809.610924,2.456717
105117,cle_res_elec_09,12/31 22:00:00,N,Exist_LowE,1348046.103,0,1.340602e+06,0.0,164269.71510,2.852918e+06,792.477099,2.404725
105118,cle_res_elec_09,12/31 23:00:00,N,Exist_LowE,1412030.685,0,1.340602e+06,0.0,91155.44883,2.843788e+06,789.941076,2.397030


In [41]:
# Get scenario and find the max
scenario1 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_01"]
scenario2 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_02"]
scenario3 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_03"]
scenario4 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_04"]
scenario5 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_05"]
scenario6 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_06"]
scenario7 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_07"]
scenario8 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_08"]
scenario9 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_09"]
scenario10 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_10"]
scenario11 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_11"]
scenario12 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_12"]

In [42]:
scenarioPeak1 = (scenario1[scenario1["PeakEnergy[J](Hourly)"] == scenario1["PeakEnergy[J](Hourly)"].max()])
scenarioPeak2 = (scenario2[scenario2["PeakEnergy[J](Hourly)"] == scenario2["PeakEnergy[J](Hourly)"].max()])
scenarioPeak3 = (scenario3[scenario3["PeakEnergy[J](Hourly)"] == scenario3["PeakEnergy[J](Hourly)"].max()])
scenarioPeak4 = (scenario4[scenario4["PeakEnergy[J](Hourly)"] == scenario4["PeakEnergy[J](Hourly)"].max()])
scenarioPeak5 = (scenario5[scenario5["PeakEnergy[J](Hourly)"] == scenario5["PeakEnergy[J](Hourly)"].max()])
scenarioPeak6 = (scenario6[scenario6["PeakEnergy[J](Hourly)"] == scenario6["PeakEnergy[J](Hourly)"].max()])
scenarioPeak7 = (scenario7[scenario7["PeakEnergy[J](Hourly)"] == scenario7["PeakEnergy[J](Hourly)"].max()])
scenarioPeak8 = (scenario8[scenario8["PeakEnergy[J](Hourly)"] == scenario8["PeakEnergy[J](Hourly)"].max()])
scenarioPeak9 = (scenario9[scenario9["PeakEnergy[J](Hourly)"] == scenario9["PeakEnergy[J](Hourly)"].max()])
scenarioPeak10 = (scenario10[scenario10["PeakEnergy[J](Hourly)"] == scenario10["PeakEnergy[J](Hourly)"].max()])
scenarioPeak11 = (scenario11[scenario11["PeakEnergy[J](Hourly)"] == scenario11["PeakEnergy[J](Hourly)"].max()])
scenarioPeak12 = (scenario12[scenario12["PeakEnergy[J](Hourly)"] == scenario12["PeakEnergy[J](Hourly)"].max()])

In [43]:
PeakDemandRes =scenarioPeak1.append([scenarioPeak2, 
                                  scenarioPeak3,
                                  scenarioPeak4,
                                  scenarioPeak5,
                                  scenarioPeak6,
                                  scenarioPeak7,
                                  scenarioPeak8,
                                  scenarioPeak9,
                                  scenarioPeak10,
                                  scenarioPeak11,
                                  scenarioPeak12])

PeakDemandRes

,Scenario,Date_Time,Elevation,GlazingSystem,HeatingElecEnergy[J](Hourly),HeatingGasEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),LightingEnergy[J](Hourly),PeakEnergy[J](Hourly),PeakEnergy(w),PeakEnergy(w/sf)
27149,cle_res_elec_01,02/06 06:00:00,N,Exist,1.224249e+07,0,1.911483e+06,0.000,93529.28864,1.424751e+07,3957.640865,12.009227
35909,cle_res_elec_02,02/06 06:00:00,E,Exist,1.320179e+07,0,9.948133e+05,0.000,93529.28864,1.429013e+07,3969.481202,12.045156
44669,cle_res_elec_03,02/06 06:00:00,S,Exist,1.227821e+07,0,1.725646e+06,0.000,93529.28864,1.409738e+07,3915.939439,11.882687
53429,cle_res_elec_04,02/06 06:00:00,W,Exist,1.224175e+07,0,1.542940e+06,0.000,93529.28864,1.387822e+07,3855.060671,11.697954
66113,cle_res_elec_05,07/19 18:00:00,N,Exist_Clear,0.000000e+00,0,1.709527e+06,8882538.080,146228.53250,1.073829e+07,2982.859228,9.051310
70949,cle_res_elec_06,02/06 06:00:00,E,Exist_Clear,8.209287e+06,0,9.182772e+05,0.000,93529.28864,9.221093e+06,2561.414826,7.772462
83626,cle_res_elec_07,07/19 11:00:00,S,Exist_Clear,0.000000e+00,0,1.518963e+06,7888369.941,28248.69378,9.435581e+06,2620.994808,7.953254
93040,cle_res_elec_08,08/15 17:00:00,W,Exist_Clear,0.000000e+00,0,1.340995e+06,7403623.962,104211.56780,8.848830e+06,2458.008411,7.458681
97229,cle_res_elec_09,02/06 06:00:00,N,Exist_LowE,8.023403e+06,0,1.340602e+06,0.000,93529.28864,9.457534e+06,2627.092916,7.971758
869,cle_res_elec_10,02/06 06:00:00,E,Exist_LowE,8.679978e+06,0,7.151005e+05,0.000,93529.28864,9.488607e+06,2635.724297,7.997950


In [44]:
# Export dataframe  to csv
PeakDemandRes.to_csv("Scraper_Output/cle_peak_res.csv", header=True, index=True)